In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medical-chatbot-dataset/validation_data_chatbot.csv
/kaggle/input/medical-chatbot-dataset/train_data_chatbot.csv


In [2]:
import json
import pandas as pd
import kagglehub

# 🔹 Step 1: Download the dataset using KaggleHub
path = kagglehub.dataset_download("saifulislamsarfaraz/medical-chatbot-dataset")

print("Path to dataset files:", path)

# 🔹 Step 2: Define the correct file paths
train_data = f"{path}/train_data_chatbot.csv"
val_data = f"{path}/validation_data_chatbot.csv"  # Assuming the validation file exists

print("Training dataset path:", train_data)
print("Validation dataset path:", val_data)

# 🔹 Step 3: Function to Convert Dataset to Training Format (Using First 80 Rows)
def convert_to_training_format(input_csv, output_file, num_rows=80):
    system_message = {
        "role": "system",
        "content": "Extract Details from medical report"
    }

    df = pd.read_csv(input_csv).head(num_rows)  # Take only the first 'num_rows' rows

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for _, row in df.iterrows():
            medical_report = f"Patient asks: {row['short_question']}"  # Simulating a medical report
            extracted_json = json.dumps({
                "answer": row['short_answer'],
                "tags": row['tags']
            })

            training_example = {
                "messages": [
                    system_message,
                    {"role": "user", "content": medical_report},
                    {"role": "assistant", "content": extracted_json}
                ]
            }
            outfile.write(json.dumps(training_example) + '\n')

# 🔹 Step 4: Convert and Save Both Training and Validation Data (First 80 Rows)
train_output_file = "formatted_training_data.jsonl"
val_output_file = "formatted_validation_data.jsonl"

convert_to_training_format(train_data, train_output_file, num_rows=80)
convert_to_training_format(val_data, val_output_file, num_rows=80)

print("Conversion complete.")
print("Training data saved to:", train_output_file)
print("Validation data saved to:", val_output_file)

Path to dataset files: /kaggle/input/medical-chatbot-dataset
Training dataset path: /kaggle/input/medical-chatbot-dataset/train_data_chatbot.csv
Validation dataset path: /kaggle/input/medical-chatbot-dataset/validation_data_chatbot.csv
Conversion complete.
Training data saved to: formatted_training_data.jsonl
Validation data saved to: formatted_validation_data.jsonl


In [3]:
import json

# Read JSONL file into a list
with open("formatted_validation_data.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Print first few entries
for entry in data[:3]:  # Change 3 to any number you want
    print(json.dumps(entry, indent=4))  # Pretty-print JSON


{
    "messages": [
        {
            "role": "system",
            "content": "Extract Details from medical report"
        },
        {
            "role": "user",
            "content": "Patient asks: do i have a yeast infection"
        },
        {
            "role": "assistant",
            "content": "{\"answer\": \"hi this can be a vaginal fungal infection which is quiet common even in women who are not sexually active this may occur due to poor vaginal hygiene insert vaginal antifungal pessaries for 3 days inside your vagina and take anti histaminic tablets orally maintain good vaginal hygiene thanks\", \"tags\": \"['yeast infection']\"}"
        }
    ]
}
{
    "messages": [
        {
            "role": "system",
            "content": "Extract Details from medical report"
        },
        {
            "role": "user",
            "content": "Patient asks: does your upper backlung area hurt badly with plural effusion  for 3 days now my mid back where my lungs are hurt

In [4]:
# make sure to use the latest version of the openai python package
!pip install --upgrade --quiet openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 9.2 MB/s eta 0:00:00:00:01


In [ ]:
from openai import OpenAI
from time import sleep


your_api_key=""
# Initialize OpenAI client
client = OpenAI(api_key = your_api_key)


In [6]:
def upload_training_file(file_path):
    """Upload training file to OpenAI"""
    with open(file_path, "rb") as file:
        response = client.files.create(
            file=file,
            purpose="fine-tune"
        )
        return response.id

# Upload both training and validation files
training_file_id = upload_training_file("formatted_training_data.jsonl")
validation_file_id = upload_training_file("formatted_validation_data.jsonl")


In [7]:
def create_fine_tuning_job(training_file_id, validation_file_id=None, model="gpt-3.5-turbo-0125"):
    """Create a fine-tuning job"""
    response = client.fine_tuning.jobs.create(
        training_file=training_file_id,
        validation_file=validation_file_id,
        model=model
    )
    return response.id
model="gpt-3.5-turbo-0125"
# Start the fine-tuning job
job_id = create_fine_tuning_job(training_file_id, validation_file_id, model)


BadRequestError: Error code: 400 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [ ]:
def monitor_job(job_id):
    """Monitor fine-tuning job progress"""
    while True:
        job = client.fine_tuning.jobs.retrieve(job_id)
        print(f"Status: {job.status}")

        if job.status in ["succeeded", "failed"]:
            return job

        # List latest events
        events = client.fine_tuning.jobs.list_events(
            fine_tuning_job_id=job_id,
            limit=5
        )
        for event in events.data:
            print(f"Event: {event.message}")

        sleep(30)  # Check every 30 seconds

# Monitor the job until completion
job = monitor_job(job_id)
if job.status == "succeeded":
    fine_tuned_model = job.fine_tuned_model
    print(f"Fine-tuned model ID: {fine_tuned_model}")
else:
    print("Fine-tuning failed.")


In [ ]:
def test_model(model_id, test_input):
    """Test the fine-tuned model"""
    completion = client.chat.completions.create(
        model=model_id,
        messages=[
            {
                "role": "system",
                "content": "Extract Details from medical report"
            },
            {"role": "user", "content": test_input}
        ]
    )
    return completion.choices[0].message


In [ ]:
# Test input
test_report = """Marcus Wong, a 19-year-old male, presents with severe acne 
on face and upper back present for 1 year. Multiple inflammatory papules 
and nodules noted on examination. Previous trials of over-the-counter 
treatments ineffective. Started on Isotretinoin 40mg daily with monthly 
liver function monitoring."""

# Get prediction
result = test_model(fine_tuned_model, test_report)

# Parse the JSON response
import json
extracted_data = json.loads(result.content)
print(json.dumps(extracted_data, indent=2))
